# Summary of the Code

This Python script creates a video from a series of images stored in subfolders. It reads images from a selected subfolder, processes them, and generates a video with specific parameters such as frames per second (fps) and image display interval. The video is saved as an AVI file.

### Key Components:

1. **Parameters:**
   - `base_output_folder`: The base folder containing subfolders with images.
   - `image_extension`: The extension of the images to process (e.g., `.jpg`).
   - `fps`: Frames per second for the output video.
   - `interval_seconds`: The display duration for each image, in seconds (e.g., 10 minutes, represented as `10 * 60` seconds).

2. **Subfolder Selection:**
   - The script lists all subfolders in the `base_output_folder` and asks the user to choose one for processing.
   - If no subfolders are found, it raises an error.

3. **Video Setup:**
   - The script constructs the output video file path based on the selected subfolder and the defined interval.
   - It determines the video resolution by reading the first image in the selected subfolder.

4. **Image Processing:**
   - It reads all the image files in the selected subfolder and ensures they match the required extension.
   - For each image, the script ensures the resolution is consistent with the first image.
   - The images are written into the video, with each image repeated for the specified number of frames (based on the `interval_seconds` and `fps`).

5. **Video Creation:**
   - It uses OpenCV's `VideoWriter` to create and save the video.
   - A progress bar (via `tqdm`) is displayed during the image processing.

6. **Error Handling:**
   - If the script cannot find any images in the subfolder or if an image fails to load, it raises appropriate errors.

### Workflow:
1. List subfolders and allow user to select one.
2. Read image files and validate.
3. Set video parameters like resolution, fps, and interval.
4. Write each image to the video, repeating it for the specified duration.
5. Save the output video and release the video writer.

### Example Output:
- If a subfolder `subfolder_name` is selected with images, the output video is saved as:

output_video_subfolder_name_600s_interval.avi


### Dependencies:
- `cv2` (OpenCV): For image reading and video writing.
- `os`: For directory handling.
- `numpy`: For image manipulation (though not directly used in this code).
- `tqdm`: For showing a progress bar during the video creation process.



In [2]:
import cv2
import os
import numpy as np
from tqdm import tqdm  # Progress bar for video creation

# Parameters
base_output_folder = "output_images"  # Folder containing subfolders of images
image_extension = ".jpg"              # Extension of the image files to process (e.g., ".jpg")
fps = 1                               # Frames per second for the output video
interval_seconds = 10 * 60            # Interval time in seconds each image is displayed (10 minutes = 600 seconds)

# List all subfolders inside the base folder, which might contain the images
subfolders = sorted([d for d in os.listdir(base_output_folder) if os.path.isdir(os.path.join(base_output_folder, d))])

# If no subfolders are found, raise an error
if not subfolders:
    raise FileNotFoundError(f"No subfolders found in {base_output_folder}.")

# Prompt user to select a subfolder from the available options
print("Select a folder to process:")
for i, subfolder in enumerate(subfolders, start=1):
    print(f"{i}. {subfolder}")

# Loop to allow the user to input a valid subfolder choice
while True:
    try:
        # Ask user to choose a subfolder by entering its number
        choice = int(input(f"Enter the number corresponding to the folder (1-{len(subfolders)}): "))
        if 1 <= choice <= len(subfolders):  # Validate the user input
            selected_subfolder = subfolders[choice - 1]  # Set the selected subfolder
            break  # Exit the loop when valid input is received
        else:
            print(f"Invalid choice. Please enter a number between 1 and {len(subfolders)}.")
    except ValueError:
        # Handle invalid input (non-numeric values)
        print("Invalid input. Please enter a valid number.")

# Build the full path to the selected subfolder containing the images
output_folder = os.path.join(base_output_folder, selected_subfolder)
print(f"Using images from: {output_folder}")

# Construct dynamic video filename based on selected folder and interval
output_video = os.path.abspath(f"output_video_{selected_subfolder}_{interval_seconds}s_interval.avi")
print(f"Saving video to: {output_video}")

# Get a sorted list of image files (with the specified extension) from the selected subfolder
image_files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(image_extension)])

# If no image files are found, raise an error
if not image_files:
    raise FileNotFoundError(f"No images with extension '{image_extension}' found in {output_folder}.")
print("Image files to process:", image_files)

# Calculate how many times each image should be repeated based on the FPS and interval
num_frames_per_image = fps * interval_seconds
print(f"Each image will be displayed for {interval_seconds} seconds ({num_frames_per_image} frames).")

# Read the first image to determine the resolution (height and width) of the video
sample_image = cv2.imread(image_files[0])
if sample_image is None:
    raise FileNotFoundError("Could not read the first image to determine resolution.")

# Extract the resolution (height and width) from the first image
frame_height, frame_width = sample_image.shape[:2]
print(f"Video resolution: {frame_width}x{frame_height}")

# Define the codec for the video and initialize the VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 'XVID' is the codec for AVI format
video_writer = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

# Process each image and write it to the video
try:
    # Iterate through each image in the image files list
    for image_file in tqdm(image_files, desc="Processing Images", unit="image"):
        image = cv2.imread(image_file)  # Read the current image
        if image is None:  # If the image cannot be read, skip to the next image
            print(f"Failed to read image: {image_file}")
            continue

        # Resize the image to match the resolution of the first image, if needed
        if (image.shape[1] != frame_width) or (image.shape[0] != frame_height):
            image = cv2.resize(image, (frame_width, frame_height), interpolation=cv2.INTER_AREA)

        # Write the image to the video multiple times, based on the number of frames per image
        for _ in range(num_frames_per_image):
            video_writer.write(image)  # Write the image as a frame to the video

        # Print a progress update with the current image and how many times it's been repeated
        tqdm.write(f"Added: {image_file} repeated {num_frames_per_image} times")

    # Once all images are processed, print the success message
    print(f"Video saved as {output_video}.")
finally:
    # Release the video writer to finalize and save the video file
    video_writer.release()


Select a folder to process:
1. Lagrange
2. output_10_2025-04-08
3. output_47_2025-04-02
4. output_47_2025-04-08
5. output_60_2025-04-08
Using images from: output_images\Lagrange
Saving video to: c:\Users\Dennis\desktop\Pennstate\Strain Reference\output_video_Lagrange_600s_interval.avi
Image files to process: ['output_images\\Lagrange\\Lagrange_strain_001.jpg', 'output_images\\Lagrange\\Lagrange_strain_002.jpg', 'output_images\\Lagrange\\Lagrange_strain_003.jpg', 'output_images\\Lagrange\\Lagrange_strain_004.jpg', 'output_images\\Lagrange\\Lagrange_strain_005.jpg', 'output_images\\Lagrange\\Lagrange_strain_006.jpg', 'output_images\\Lagrange\\Lagrange_strain_007.jpg', 'output_images\\Lagrange\\Lagrange_strain_008.jpg', 'output_images\\Lagrange\\Lagrange_strain_009.jpg', 'output_images\\Lagrange\\Lagrange_strain_010.jpg', 'output_images\\Lagrange\\Lagrange_strain_011.jpg', 'output_images\\Lagrange\\Lagrange_strain_012.jpg', 'output_images\\Lagrange\\Lagrange_strain_013.jpg', 'output_image

Processing Images:   2%|▏         | 1/42 [00:07<04:52,  7.13s/image]

Added: output_images\Lagrange\Lagrange_strain_001.jpg repeated 600 times


Processing Images:   5%|▍         | 2/42 [00:14<04:44,  7.11s/image]

Added: output_images\Lagrange\Lagrange_strain_002.jpg repeated 600 times


...

Added: output_images\Lagrange\Lagrange_strain_011.jpg repeated 600 times


Processing Images:  29%|██▊       | 12/42 [01:26<03:41,  7.40s/image]

Added: output_images\Lagrange\Lagrange_strain_012.jpg repeated 600 times


Processing Images:  29%|██▊       | 12/42 [01:30<03:45,  7.53s/image]


KeyboardInterrupt: 